In [ ]:
from src.data.data import *
from src.orcml import *
from src.plotting import *
from src.utils.graph_utils import *
from src.isorc import *
from src.utils.embeddings import *
from sklearn.manifold import TSNE
import umap
import numpy as np

exp_params = {
    'mode': 'nbrs',
    'n_neighbors': 25,
    'epsilon': None,
    'lda': 1e-5,
    'delta': 0.8
}

%load_ext autoreload

In [ ]:
######### Edge length scaling vs ORC ###############

# compute edge-wise length scaling in G
def plot_edge_length_scaling(G, ref, data, orcs):
    edge_lengths_ref = []
    edge_lengths_data = []
    for edge in G.edges:
        edge_lengths_ref.append(np.linalg.norm(ref[edge[0]] - ref[edge[1]]))
        edge_lengths_data.append(np.linalg.norm(data[edge[0]] - data[edge[1]]))
    edge_lengths_ref = np.array(edge_lengths_ref)
    edge_lengths_data = np.array(edge_lengths_data)
    length_scaling = edge_lengths_data / edge_lengths_ref

    # compute edge-wise length scaling in G
    orcs = np.array(orcs)
    # plot edge length scaling vs ORC
    plt.figure()
    plt.scatter(length_scaling, orcs, s=2)
    plt.xlabel('Edge length scaling factor')
    plt.ylabel('ORC')
    plt.title('Edge length scaling vs ORC for UMAP embedding')

####### quadratics #######
n_points = 2000
noise = 0.175
noise_thresh = None
return_dict = quadratics(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=True)
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
plot_data_2D(return_dict['data'], color=None, title=None, node_size=3)
plt.title('Original Data')
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
G = orcmanl.G.copy()
umap_emb = umap.UMAP(n_neighbors=25, min_dist=0.1, metric='euclidean').fit_transform(return_dict['data'])
plot_graph_2D(umap_emb, G, node_color=None, edge_color=orcmanl.orcs, title=None, edge_width=0.05, node_size=0.0)
plt.title('UMAP Embedding with annotated edge ORC')
plot_edge_length_scaling(G, return_dict['data'], umap_emb, orcmanl.orcs)


####### circles #######
n_points = 2000
noise = 0.1
noise_thresh = None
return_dict = concentric_circles(n_points=n_points, factor=0.4, noise=noise, noise_thresh=noise_thresh, supersample=True)
plot_data_2D(return_dict['data'], color=None, title=None, node_size=3)
plt.title('Original Data')
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
G = orcmanl.G.copy()
umap_emb = umap.UMAP(n_neighbors=25, min_dist=0.1, metric='euclidean').fit_transform(return_dict['data'])
plot_graph_2D(umap_emb, G, node_color=None, edge_color=orcmanl.orcs, title=None, edge_width=0.05, node_size=0.0)
plt.title('UMAP Embedding with annotated edge ORC')
plot_edge_length_scaling(G, return_dict['data'], umap_emb, orcmanl.orcs)

####### swiss roll #######
n_points = 2000
noise = 0.5
noise_thresh = None
return_dict = swiss_roll(n_points=n_points, noise=noise, noise_thresh=noise_thresh, supersample=False)
orcmanl = ORCManL(
    exp_params=exp_params,
    verbose=True,
    reattach=True,
)
orcmanl.fit(return_dict['data'])
G = orcmanl.G.copy()
# rotate data about z axis by 15 degrees
R = np.array([[np.cos(np.pi/12), -np.sin(np.pi/12), 0], [np.sin(np.pi/12), np.cos(np.pi/12), 0], [0, 0, 1]])
X = np.dot(return_dict['data'], R)
plt.figure()
plt.scatter(X[:, 0], X[:, 2], s=2)
# turn off axes
plt.axis('off')
plt.title('Original Data')
umap_emb = umap.UMAP(n_neighbors=25, min_dist=0.1, metric='euclidean').fit_transform(return_dict['data'])
plot_graph_2D(umap_emb, G, node_color=None, edge_color=orcmanl.orcs, title=None, edge_width=0.1, node_size=0.0)
plt.title('UMAP Embedding with annotated edge ORC')
plot_edge_length_scaling(G, return_dict['data'], umap_emb, orcmanl.orcs)